In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
LEARNING_RATE = 3e-3 #0.14
EPOCHS = 25
TARGET_FOLDER = "weights"
K_FOLDS = 3

In [5]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from sklearn.model_selection import KFold

In [10]:
transform = transforms.Compose([transforms.Resize((256, 256)),
                                 transforms.ToTensor()])

data_set = datasets.ImageFolder('/content/drive/MyDrive/data_aml/X-Ray Image DataSet', transform=transform)


In [5]:
data_loader = torch.utils.data.DataLoader(data_set, batch_size=32, shuffle=True)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Current device: {device}")

Current device: cuda:0


In [18]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super(ConvBlock, self).__init__()
        for_pad = lambda s: s if s > 2 else 3
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=(for_pad(kernel_size) - 1)//2, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True) 

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out

class TripleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TripleConvBlock, self).__init__()
        self.conv_block_1 = ConvBlock(in_channels, out_channels)
        self.conv_block_2 = ConvBlock(out_channels, in_channels, kernel_size=1)  
        self.conv_block_3 = ConvBlock(in_channels, out_channels)

    def forward(self, x):
        out = self.conv_block_1(x)
        out = self.conv_block_2(out)
        out = self.conv_block_3(out)
        return out

class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.seq = nn.Sequential(
        ConvBlock(3, 8),
        nn.MaxPool2d(2, stride=2),
        ConvBlock(8, 16),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(16, 32),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(32,64),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(64,128),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(128,256),
        ConvBlock(256, 128, kernel_size=1),
        ConvBlock(128, 256),
        nn.Conv2d(256, 2, 3, padding=(3-1)//2, stride=1), # Ich bekomme es nicht hin, dass das identisch ist 
        nn.ReLU(),
        nn.BatchNorm2d(2),
        nn.Flatten(),
        nn.Linear(338,2)
        )

    def forward(self, x):
        return self.seq(x)

class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.seq = nn.Sequential(
        ConvBlock(3, 8),
        nn.MaxPool2d(2, stride=2),
        ConvBlock(8, 16),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(16, 32),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(32,64),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(64,128),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(128,256),
        ConvBlock(256, 128, kernel_size=1),
        ConvBlock(128, 256),
        nn.Conv2d(256, 3, 3, padding=(3-1)//2, stride=1), # Ich bekomme es nicht hin, dass das identisch ist 
        nn.ReLU(),
        nn.BatchNorm2d(3),
        nn.Flatten(),
        nn.Linear(507,3)
        )

    def forward(self, x):
        return self.seq(x)

In [8]:
from torchsummary import summary
model = Model3().to(device)
summary(model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 256, 256]             216
       BatchNorm2d-2          [-1, 8, 256, 256]              16
         LeakyReLU-3          [-1, 8, 256, 256]               0
         ConvBlock-4          [-1, 8, 256, 256]               0
         MaxPool2d-5          [-1, 8, 128, 128]               0
            Conv2d-6         [-1, 16, 128, 128]           1,152
       BatchNorm2d-7         [-1, 16, 128, 128]              32
         LeakyReLU-8         [-1, 16, 128, 128]               0
         ConvBlock-9         [-1, 16, 128, 128]               0
        MaxPool2d-10           [-1, 16, 64, 64]               0
           Conv2d-11           [-1, 32, 64, 64]           4,608
      BatchNorm2d-12           [-1, 32, 64, 64]              64
        LeakyReLU-13           [-1, 32, 64, 64]               0
        ConvBlock-14           [-1, 32,

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [9]:
def train(model, data_loader, optimizer):
    """
    model -- neural net
    data_loader -- dataloader for train images
    optimizer -- optimizer
    """
    model.train()
    
    criterion = torch.nn.CrossEntropyLoss().to(device)
    
    for step, [images, labels] in enumerate(data_loader,0):
        images = images.to(device)
        labels = labels.to(device)
                
        optimizer.zero_grad()
        result = model(images)
        loss = criterion(result, labels)
                
        # backpropagation
        loss.backward()
        optimizer.step()
                                    
        if step%10 == 0:
            print(f"Step: {step}, loss: {loss}")

In [21]:
def calc_accuracy(result, labels):
    result = torch.round(result)
    probs = torch.softmax(result, dim=1)

    correct_results_sum = (probs.argmax(dim=1) == labels).sum().float()
    acc = correct_results_sum/labels.shape[0]
    acc = torch.round(acc * 100)
    
    return acc


In [20]:
def test(model, test_loader, epoch):
    """    
    model -- neural net 
    test_loader -- dataloader of test images
    epoch -- current epoch
    """
    model.eval()
    loss = 0
    accuracy = 0
    criterion = torch.nn.CrossEntropyLoss().to(device)

    for step, [images, labels] in enumerate(test_loader,0):
        images = images.to(device)
        labels = labels.to(device)

        result = model(images)
        loss += criterion(result.detach(), labels.detach())
        accuracy += calc_accuracy(result.detach(), labels.detach())
    loss /= step
    accuracy /=  step
  
    print(f"Loss: {loss}, Accuracy: {accuracy}")

In [16]:
kfold = KFold(n_splits=K_FOLDS, shuffle=True)

In [14]:
for fold, (train_ids, test_ids) in enumerate(kfold.split(data_set)):
    
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    train_loader = torch.utils.data.DataLoader(
                      data_set, 
                      batch_size=32, sampler=train_subsampler)
    test_loader = torch.utils.data.DataLoader(
                      data_set,
                      batch_size=32, sampler=test_subsampler)
    
    model = Model3().to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
    
    for epoch in range(0, EPOCHS):

      train(model, train_loader, optimizer)
      
      #torch.save(model.state_dict(), f'{TARGET_FOLDER}/fold_{fold}_epoch_{epoch}.ckpt')

      test(model, test_loader, epoch)

FOLD 0
--------------------------------
Step: 0, loss: 1.1709269285202026
Step: 10, loss: 0.85526043176651
Step: 20, loss: 0.8224045634269714
Loss: 1.5488566160202026, Accuracy: 8.545454978942871
Step: 0, loss: 0.7802308201789856
Step: 10, loss: 0.6580170392990112
Step: 20, loss: 0.5885074734687805
Loss: 0.8503888249397278, Accuracy: 72.36363983154297
Step: 0, loss: 0.5039673447608948
Step: 10, loss: 0.4858916401863098
Step: 20, loss: 0.6375385522842407
Loss: 0.6914443969726562, Accuracy: 77.81818389892578
Step: 0, loss: 0.5469644069671631
Step: 10, loss: 0.7296732664108276
Step: 20, loss: 0.5378980040550232
Loss: 0.7396879196166992, Accuracy: 63.54545593261719
Step: 0, loss: 0.5758719444274902
Step: 10, loss: 0.5874923467636108
Step: 20, loss: 0.7288925647735596
Loss: 0.8652558922767639, Accuracy: 75.18182373046875
Step: 0, loss: 0.6100776195526123
Step: 10, loss: 0.547454833984375
Step: 20, loss: 0.38228917121887207
Loss: 0.8177903294563293, Accuracy: 77.54545593261719
Step: 0, loss:

# Federated
https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/

Neuste Version (0.3.x) hat kein TorchHook -> Lösung finden?

In [15]:
!pip install syft==0.2.9 

     |████████████████████████████████| 433 kB 8.7 MB/s 
     |████████████████████████████████| 20.1 MB 8.2 MB/s 
     |████████████████████████████████| 66 kB 5.9 MB/s 
     |████████████████████████████████| 200 kB 78.2 MB/s 
     |████████████████████████████████| 2.0 MB 45.4 MB/s 
     |████████████████████████████████| 484 kB 73.0 MB/s 
     |████████████████████████████████| 4.0 MB 30.4 MB/s 
     |████████████████████████████████| 57 kB 6.5 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 1.8 MB 60.4 MB/s 
     |████████████████████████████████| 79 kB 9.5 MB/s 
     |████████████████████████████████| 9.0 MB 65.6 MB/s 
     |████████████████████████████████| 73 kB 2.6 MB/s 
     |████████████████████████████████| 126 kB 70.2 MB/s 
     |████████████████████████████████| 753.4 MB 7.0 kB/s 
     |████████████████████████████████| 449 kB 63.8 MB/s 
     |████████████████████████████████| 3.5 MB 26.0 MB/s 
     |██████████████████

In [1]:
import syft as sy

In [7]:
hook = sy.TorchHook(torch)
nr_of_instances = 4
instances = []
for i in range(nr_of_instances):
  instances.append(sy.VirtualWorker(hook, id=str(i)))

In [12]:
class Arguments():
    def __init__(self):
        self.batch_size = 32
        self.test_batch_size = 1000
        self.epochs = 25
        self.lr = 3e-3
        self.seed = 1
        self.log_interval = 10
        self.save_model = False
args = Arguments()

torch.manual_seed(args.seed)
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

In [13]:
federated_data_loader = sy.FederatedDataLoader(data_set.federate(instances),
    batch_size=args.batch_size, shuffle=True, **kwargs)

In [23]:
def train(args, model, fed_train_loader, optims, epoch):
    model.train()

    criterion = torch.nn.CrossEntropyLoss().to(device)

    for batch_idx, (data, target) in enumerate(fed_train_loader): # <-- now it is a distributed dataset
        model.send(data.location)
        data, target = data.to(device), target.to(device)

        optimizer = optims.get_optim(data.location.id)
        optimizer.zero_grad()
        
        output = model(data)

        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

        model.get()
        if batch_idx % args.log_interval == 0:
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(fed_train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(fed_train_loader), loss.item()))


In [24]:

from syft.federated.floptimizer import Optims

torch.set_default_tensor_type(torch.cuda.FloatTensor)

for fold, (train_ids, test_ids) in enumerate(kfold.split(data_set)):
    
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    test_loader = torch.utils.data.DataLoader(
                      data_set,
                      batch_size=32, sampler=test_subsampler)
    
    federated_model = Model3().to(device)
    
    federated_optimizer = Optims(list(range(nr_of_instances)), optim=torch.optim.Adam(params=federated_model.parameters(),lr=args.lr))
    
    for epoch in range(1, args.epochs + 1):

      train(args, federated_model, federated_data_loader, federated_optimizer, epoch)

      if (args.save_model):
        torch.save(federated_model.state_dict(), f'{TARGET_FOLDER}/federated_fold_{fold}_epoch_{epoch}.ckpt')

      test(federated_model, test_loader, epoch)



torch.set_default_tensor_type(torch.FloatTensor)


FOLD 0
--------------------------------
Train Epoch: 1 [0/1152 (0%)]	Loss: 1.030318
Train Epoch: 1 [320/1152 (28%)]	Loss: 0.022952
Train Epoch: 1 [640/1152 (56%)]	Loss: 1.238003
Train Epoch: 1 [960/1152 (83%)]	Loss: 0.000879
Loss: 7.61448335647583, Accuracy: 43.181819915771484
Train Epoch: 2 [0/1152 (0%)]	Loss: 10.793051
Train Epoch: 2 [320/1152 (28%)]	Loss: 0.280800
Train Epoch: 2 [640/1152 (56%)]	Loss: 2.237763
Train Epoch: 2 [960/1152 (83%)]	Loss: 0.005692
Loss: 6.579258441925049, Accuracy: 43.090911865234375
Train Epoch: 3 [0/1152 (0%)]	Loss: 9.422319
Train Epoch: 3 [320/1152 (28%)]	Loss: 0.355277
Train Epoch: 3 [640/1152 (56%)]	Loss: 5.091043
Train Epoch: 3 [960/1152 (83%)]	Loss: 0.005065
Loss: 6.6749796867370605, Accuracy: 43.3636360168457
Train Epoch: 4 [0/1152 (0%)]	Loss: 8.836844
Train Epoch: 4 [320/1152 (28%)]	Loss: 0.323474
Train Epoch: 4 [640/1152 (56%)]	Loss: 4.819616
Train Epoch: 4 [960/1152 (83%)]	Loss: 0.023365
Loss: 4.348712921142578, Accuracy: 43.090911865234375
Train